# Data Loading

The data set is provided by Ben Roshan, published in [Kaggle](https://www.kaggle.com/benroshan/ecommerce-data). 


We have 3 data sets in CSV format. Spark supports loading CSV files and infer schemas directly. Also, there is a flag to flag whether the files have a header row.

In [0]:
sales_target = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/shared_uploads/dtuworkshop@outlook.com/Sales_target.csv")
sales_order_header = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/shared_uploads/dtuworkshop@outlook.com/List_of_Orders.csv")
sales_order_detail = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("dbfs:/FileStore/shared_uploads/dtuworkshop@outlook.com/Order_Details.csv")

# Sales Order Header Data

There are some data processing needed for this data set:

1. Records with missing ID
2. Order Date in String format
3. We prefer snake_case column names

In [0]:
display(sales_order_header)

Order ID,Order Date,CustomerName,State,City
B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
B-25602,01-04-2018,Pearl,Maharashtra,Pune
B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
B-25605,05-04-2018,Kasheen,West Bengal,Kolkata
B-25606,06-04-2018,Hazel,Karnataka,Bangalore
B-25607,06-04-2018,Sonakshi,Jammu and Kashmir,Kashmir
B-25608,08-04-2018,Aarushi,Tamil Nadu,Chennai
B-25609,09-04-2018,Jitesh,Uttar Pradesh,Lucknow
B-25610,09-04-2018,Yogesh,Bihar,Patna


In [0]:
import pyspark.sql.functions as fn

sales_order_header = sales_order_header.dropna("any")
sales_order_header = sales_order_header.withColumnRenamed(
  "Order ID", "order_id"
).withColumn(
  "order_date",
  fn.to_date(sales_order_header["Order Date"], "d-M-y")
).withColumnRenamed(
  "State", "state"
).withColumnRenamed(
  "City", "city"
).withColumnRenamed(
  "CustomerName", "customer_name"
)
sales_order_header = sales_order_header.drop("Order Date")

In [0]:
display(sales_order_header)

order_id,customer_name,state,city,order_date
B-25601,Bharat,Gujarat,Ahmedabad,2018-04-01
B-25602,Pearl,Maharashtra,Pune,2018-04-01
B-25603,Jahan,Madhya Pradesh,Bhopal,2018-04-03
B-25604,Divsha,Rajasthan,Jaipur,2018-04-03
B-25605,Kasheen,West Bengal,Kolkata,2018-04-05
B-25606,Hazel,Karnataka,Bangalore,2018-04-06
B-25607,Sonakshi,Jammu and Kashmir,Kashmir,2018-04-06
B-25608,Aarushi,Tamil Nadu,Chennai,2018-04-08
B-25609,Jitesh,Uttar Pradesh,Lucknow,2018-04-09
B-25610,Yogesh,Bihar,Patna,2018-04-09


In [0]:
sales_order_header.write.format("delta").saveAsTable("sales_order_header")

# Sales Order Detail

There are 1 data processing needed for this data set:

1. We prefer snake_case column names

In [0]:
sales_order_detail = sales_order_detail.withColumnRenamed(
  "Order ID", "order_id"
).withColumnRenamed(
  "Amount", "amount"
).withColumnRenamed(
  "Profit","profit"
).withColumnRenamed(
  "Quantity", "quantity"
).withColumnRenamed(
  "Category", "category"
).withColumnRenamed(
  "Sub-Category", "sub_category"
)

In [0]:
display(sales_order_detail)

order_id,amount,profit,quantity,category,sub_category
B-25601,1275.0,-1148.0,7,Furniture,Bookcases
B-25601,66.0,-12.0,5,Clothing,Stole
B-25601,8.0,-2.0,3,Clothing,Hankerchief
B-25601,80.0,-56.0,4,Electronics,Electronic Games
B-25602,168.0,-111.0,2,Electronics,Phones
B-25602,424.0,-272.0,5,Electronics,Phones
B-25602,2617.0,1151.0,4,Electronics,Phones
B-25602,561.0,212.0,3,Clothing,Saree
B-25602,119.0,-5.0,8,Clothing,Saree
B-25603,1355.0,-60.0,5,Clothing,Trousers


In [0]:
sales_order_detail.write.format("delta").saveAsTable("sales_order_detail")

# Sales Target Data

There are 1 data processing needed for this data set:

1. The 'Month of Order Date' column is in String, and it will make it hard to make calculations

In [0]:
import pyspark.sql.functions as fn

sales_target = sales_target.withColumn(
  "month",
  fn.from_unixtime(
    fn.unix_timestamp(
      fn.split(sales_target["Month of Order Date"], "-").getItem(0),
      'MMM'
    ),
    'M'
  ).cast("int")
).withColumn(
  "year",
  fn.split(sales_target["Month of Order Date"], "-").getItem(1).cast("int") + 2000
).withColumn(
  "target",
  sales_target["Target"].cast("double")
).withColumnRenamed(
  "Category", "category"
)
sales_target = sales_target.drop("Month of Order Date")

In [0]:
display(sales_target.sort("month"))

category,target,month,year
Furniture,11500.0,1,2019
Electronics,16000.0,1,2019
Clothing,16000.0,1,2019
Furniture,11600.0,2,2019
Electronics,16000.0,2,2019
Clothing,16000.0,2,2019
Furniture,11800.0,3,2019
Clothing,16000.0,3,2019
Electronics,16000.0,3,2019
Electronics,9000.0,4,2018


In [0]:
sales_target.write.format("delta").saveAsTable("sales_target")

# Analysis

In [0]:
%sql

SELECT a.*, b.city 
FROM sales_order_detail a
JOIN sales_order_header b
ON a.order_id = b.order_id

order_id,amount,profit,quantity,category,sub_category,city
B-25601,1275.0,-1148.0,null,Furniture,Bookcases,Ahmedabad
B-25601,66.0,-12.0,null,Clothing,Stole,Ahmedabad
B-25601,8.0,-2.0,null,Clothing,Hankerchief,Ahmedabad
B-25601,80.0,-56.0,null,Electronics,Electronic Games,Ahmedabad
B-25602,168.0,-111.0,null,Electronics,Phones,Pune
B-25602,424.0,-272.0,null,Electronics,Phones,Pune
B-25602,2617.0,1151.0,null,Electronics,Phones,Pune
B-25602,561.0,212.0,null,Clothing,Saree,Pune
B-25602,119.0,-5.0,null,Clothing,Saree,Pune
B-25603,1355.0,-60.0,null,Clothing,Trousers,Bhopal


In [0]:
%sql

SELECT b.city, sum(profit) profit, sum(amount) amount
FROM sales_order_detail a
JOIN sales_order_header b
ON a.order_id = b.order_id
GROUP BY 1
ORDER BY profit

city,profit,amount
Chennai,-2216.0,6087.0
Ahmedabad,-880.0,14230.0
Jaipur,-753.0,10076.0
Hyderabad,-496.0,13256.0
Patna,-321.0,12943.0
Kashmir,8.0,10829.0
Kohima,148.0,11903.0
Lucknow,156.0,5502.0
Chandigarh,172.0,21142.0
Goa,370.0,6705.0


In [0]:
%sql

SELECT b.city, sum(profit) / sum(amount) profit_per_amount
FROM sales_order_detail a
JOIN sales_order_header b
ON a.order_id = b.order_id
GROUP BY 1
ORDER BY profit_per_amount

city,profit_per_amount
Chennai,-0.3640545424675538
Jaipur,-0.07473203652242953
Ahmedabad,-0.06184118060435699
Hyderabad,-0.03741701870850935
Patna,-0.024801050761029127
Kashmir,7.387570412780497E-4
Chandigarh,0.008135464951281809
Kohima,0.012433840208350836
Mumbai,0.02645998674576107
Lucknow,0.028353326063249727


In [0]:
%sql

SELECT category, sub_category, sum(profit) profit
FROM sales_order_detail a
GROUP BY category, sub_category
ORDER BY profit

category,sub_category,profit
Furniture,Tables,-4011.0
Electronics,Electronic Games,-1236.0
Clothing,Kurti,181.0
Clothing,Skirt,235.0
Clothing,Leggings,260.0
Clothing,Saree,352.0
Furniture,Chairs,577.0
Furniture,Furnishings,844.0
Clothing,Shirt,1131.0
Clothing,T-shirt,1500.0


In [0]:
%sql

SELECT category, month(order_date) month, sum(profit) profit
FROM sales_order_detail a
JOIN sales_order_header b
ON a.order_id = b.order_id
GROUP BY 1, 2
ORDER BY month, category

category,month,profit
Clothing,1,1691.0
Electronics,1,4785.0
Furniture,1,3284.0
Clothing,2,1822.0
Electronics,2,1927.0
Furniture,2,2168.0
Clothing,3,5060.0
Electronics,3,3430.0
Furniture,3,1587.0
Clothing,4,-184.0
